<a href="https://www.kaggle.com/code/piyushjain572/sklearn-pipelines?scriptVersionId=199317265" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv('/kaggle/input/modified-titanic-dataset/train.csv')
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
51,52,0,3,"Nosworthy, Mr. Richard Cater",male,21.0,0,0,A/4. 39886,7.8000,NaN,S
824,825,0,3,"Panula, Master. Urho Abraham",male,2.0,4,1,3101295,39.6875,NaN,S
742,743,1,1,"Ryerson, Miss. Susan Parker ""Suzette""",female,21.0,2,2,PC 17608,262.3750,B57 B59 B63 B66,C
467,468,0,1,"Smart, Mr. John Montgomery",male,56.0,0,0,113792,26.5500,NaN,S


# Chalk out your plan for pipeline

In [5]:
# Step1: Dropping unnecessry features
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [35]:
# Step2: train test split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,1:],
                                                df.iloc[:,0],
                                                test_size=0.2,
                                                random_state=2)
print(X_train.shape)
X_train.head()

(712, 7)


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S


In [12]:
y_train.sample(5)

102    0
318    1
434    0
537    1
119    0
Name: Survived, dtype: int64

### **Note:** *Use index instead of name of cols as after transformations only arrays will be left without col name.*

In [36]:
#Imputation transformer
trf1 = ColumnTransformer(transformers=[
    ('age_imputer',SimpleImputer(),[2]),
    ('embarked_imputer',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

### **Note:** *While doing transformations, index of cols will be changed, so make sure you are using correct indices for your next transformation.*

In [37]:
#One Hot Encoding
trf2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,drop='first',handle_unknown='ignore'),[1,3])
],remainder='passthrough')

In [38]:
#Scaling
trf3 = ColumnTransformer(transformers=[
    ('scale',MinMaxScaler(),slice(0,8))
],remainder='passthrough')

In [39]:
#Feature Selection
trf4 = SelectKBest(score_func=chi2,k=6)

In [40]:
#Train the model
trf5 = DecisionTreeClassifier()

# Creating Pipeline

In [41]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

# Pipeline Vs make_pipeline
Pipeline requires naming of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

In [ ]:
#Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [42]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age_imputer',
                                                  SimpleImputer(), [2]),
                                                 ('embarked_imputer',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 3])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 8, None))])),
                ('trf4',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x7bc6beb45ab0>)),
                ('trf5', DecisionTreeClassifier())])

# Explore the Piepline

In [43]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('age_imputer', SimpleImputer(), [2]),
                                 ('embarked_imputer',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 3])]),
 'trf3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), slice(0, 8, None))]),
 'trf4': SelectKBest(k=6, score_func=<function chi2 at 0x7bc6beb45ab0>),
 'trf5': DecisionTreeClassifier()}

In [44]:
#predict
y_pred = pipe.predict(X_test)

In [45]:
y_pred

array([1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0])

In [46]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.7541899441340782


# Cross Validation using Pipeline

In [49]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy')

array([0.84615385, 0.86713287, 0.8028169 , 0.76056338, 0.75352113])

In [50]:
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

0.800413670836206

# GridSearch using Pipeline

In [53]:
#gridsearch params
params = {
    'trf5__max_depth':[1,2,3,4,10]
}
# Note: Here in pipeline, first give name of transformer(trf5)  and then param name(max_depth).

In [59]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,scoring='accuracy',cv=5)
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('age_imputer',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('embarked_imputer',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          3])])),
                                       ('trf3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 8, None))])),
                                       ('trf4',
                                        SelectKBest(k=6,
                                                    score_func=<function chi2 at 0x7bc6beb45ab0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 10]},
             scoring='accuracy')

In [60]:
grid.best_score_

0.8145966709346991

In [61]:
grid.best_params_

{'trf5__max_depth': 3}

# Exporting a Pipeline

In [ ]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

# Predicting using Exported Pipeline

In [ ]:
pipe = pickle.load(open('pipe.pkl','rb'))

In [57]:
# Assume user input
test_input2 = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [62]:
pipe.predict(test_input2)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])